In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

In [2]:
directory = os.getcwd()
data = '/data/vectors/data0625/'
path = directory + data

In [3]:
fft = 2048

In [4]:
data = np.empty((0,fft+1))
for fname in ['wp.txt','nowp.txt','silence.txt']:
    print(fname)
    with open(path+fname) as f:
        dat = np.genfromtxt(f, delimiter=',', dtype='int', 
                            invalid_raise=False)
        if fname.startswith('w'):
            dat[:,-1] = 2
        elif fname.startswith('no'):
            dat[:,-1] = 1
        else:
            dat[:,-1] = 0 ## silence
    data = np.concatenate([data,dat],axis=0)     

wp.txt
nowp.txt
silence.txt


In [5]:
data.shape

(1308, 2049)

In [6]:
data_set = data.copy()

## Split train/validation/test

In [7]:
train_len = int(.6*(data_set.shape[0]))
val_len = int(.2*(data_set.shape[0]))
np.random.seed(21)
np.random.shuffle(data_set)
trainNN, valNN, testNN = np.split(data_set, [train_len,train_len+val_len])

X_trainNN = trainNN[:,:-1]
y_trainNN = trainNN[:,-1]
X_valNN = valNN[:,:-1]
y_valNN = valNN[:,-1]
X_testNN = testNN[:,:-1]
y_testNN = testNN[:,-1]

print('number of sample: '+ str(len(X_trainNN)))
print('number of sample: '+ str(len(y_valNN)))
print('number of sample: '+ str(len(y_testNN)))

number of sample: 784
number of sample: 261
number of sample: 263


## modeling

In [8]:
import tensorflow_model_optimization as tfmot

In [14]:
tf.keras.backend.clear_session()

In [15]:
model = keras.Sequential()
model.add(keras.Input(shape=(fft,)))
model.add(keras.layers.Dropout(.1))
model.add(keras.layers.Dense(32, activation='relu'))
model.add(keras.layers.Dropout(.1))
model.add(keras.layers.Dense(12, activation='relu'))
model.add(keras.layers.Dense(3, activation='softmax'))

In [16]:
model = tfmot.quantization.keras.quantize_model(model)
model.compile(loss = 'sparse_categorical_crossentropy',
              optimizer = 'adam',
              metrics = ['acc'])

In [17]:
best_model_path = './models/TFLite/0625/threeClass/QAW_Integer.h5'

es = keras.callbacks.EarlyStopping(monitor='val_loss', 
                                   mode='min', 
                                   verbose=0, 
                                   patience=10)
mc = keras.callbacks.ModelCheckpoint(filepath=best_model_path, 
                                     monitor='val_acc', 
                                     mode='max', 
                                     verbose=0, 
                                     save_best_only=True)


In [18]:
history = model.fit(X_trainNN,
                    y_trainNN, 
                    epochs=500,
                    batch_size=40,
                    validation_data=(X_valNN, y_valNN),
                    callbacks=[mc,es])

Epoch 1/500
20/20 [==============================] - 0s 8ms/step - loss: 0.7011 - acc: 0.6773 - val_loss: 0.4866 - val_acc: 0.7663
Epoch 2/500
20/20 [==============================] - 0s 3ms/step - loss: 0.3850 - acc: 0.8329 - val_loss: 0.4015 - val_acc: 0.8008
Epoch 3/500
20/20 [==============================] - 0s 3ms/step - loss: 0.2717 - acc: 0.9082 - val_loss: 0.2458 - val_acc: 0.9272
Epoch 4/500
20/20 [==============================] - 0s 3ms/step - loss: 0.1830 - acc: 0.9490 - val_loss: 0.1867 - val_acc: 0.9425
Epoch 5/500
20/20 [==============================] - 0s 2ms/step - loss: 0.1203 - acc: 0.9694 - val_loss: 0.1667 - val_acc: 0.9349
Epoch 6/500
20/20 [==============================] - 0s 3ms/step - loss: 0.0829 - acc: 0.9872 - val_loss: 0.1352 - val_acc: 0.9464
Epoch 7/500
20/20 [==============================] - 0s 3ms/step - loss: 0.0562 - acc: 0.9936 - val_loss: 0.1189 - val_acc: 0.9617
Epoch 8/500
20/20 [==============================] - 0s 3ms/step - loss: 0.0299 - a

In [19]:
test_loss = model.evaluate(X_testNN, y_testNN)

9/9 [==============================] - 0s 746us/step - loss: 0.1277 - acc: 0.9658


In [20]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_testNN, np.argmax(model.predict(X_testNN), axis=-1))

array([[97,  0,  0],
       [ 0, 77,  4],
       [ 0,  5, 80]])

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
quantize_layer (QuantizeLaye (None, 2048)              3         
_________________________________________________________________
quant_dropout (QuantizeWrapp (None, 2048)              1         
_________________________________________________________________
quant_dense (QuantizeWrapper (None, 32)                65573     
_________________________________________________________________
quant_dropout_1 (QuantizeWra (None, 32)                1         
_________________________________________________________________
quant_dense_1 (QuantizeWrapp (None, 12)                401       
_________________________________________________________________
quant_dense_2 (QuantizeWrapp (None, 3)                 44        
Total params: 66,023
Trainable params: 66,003
Non-trainable params: 20
___________________________________________________

## Model Integer Post-Model Quantization

In [22]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
quantized_tflite_model = converter.convert()

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: /var/folders/7m/7gjw27ys7vs7gkgvbdgx5rjc0000gn/T/tmponejfkb5/assets


In [23]:
filename = './models/TFLite/0625/threeClass/QAW_Quantized.tflite'
with open(filename, 'wb') as f:
    f.write(quantized_tflite_model)

In [24]:
model_size = os.path.getsize(best_model_path)
print("Original model is %d KB" % int(model_size/1000))

quantized_model_size = os.path.getsize(filename)
print("Quantized model is %d KB" % int(quantized_model_size/1000))

Original model is 844 KB
Quantized model is 69 KB


## Validate the Converted Model

In [25]:
# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path=filename) 
interpreter.get_tensor_details()

[{'name': 'input_1',
  'index': 0,
  'shape': array([   1, 2048], dtype=int32),
  'shape_signature': array([  -1, 2048], dtype=int32),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}},
 {'name': 'sequential/quant_dense/BiasAdd/ReadVariableOp/resource',
  'index': 1,
  'shape': array([32], dtype=int32),
  'shape_signature': array([32], dtype=int32),
  'dtype': numpy.int32,
  'quantization': (0.00012086482456652448, 0),
  'quantization_parameters': {'scales': array([0.00012086], dtype=float32),
   'zero_points': array([0], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}},
 {'name': 'sequential/quant_dense_1/BiasAdd/ReadVariableOp/resource',
  'index': 2,
  'shape': array([12], dtype=int32),
  'shape_signature': array([12], dtype=int32),
  'dtype': numpy.int32,
  'quantization': (0.00071955996099

## Inference using Full-Quantized TFLite Model

In [26]:
interpreter = tf.lite.Interpreter(model_content=quantized_tflite_model)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()[0]
output_details = interpreter.get_output_details()[0]
scale, zero_points = input_details['quantization']

tflite_predict = []
tflite_predict_class = []
for vector in X_testNN:
    #input_data = np.array(vector.reshape(input_details['shape'])/scale + zero_points, dtype=np.int8)
    input_data = np.array(vector.reshape(input_details['shape']), dtype=np.float32)
    interpreter.set_tensor(input_details['index'], input_data)
    
    interpreter.invoke()    
    prediction = interpreter.get_tensor(output_details['index'])
    tflite_predict.append(prediction)
    tflite_predict_class.append(np.argmax(prediction))

### Performance of both the model and its optimized version are similar

In [27]:
confusion_matrix(y_testNN, tflite_predict_class)

array([[97,  0,  0],
       [ 0, 77,  4],
       [ 0,  5, 80]])